In [24]:
import psycopg2
import arcpy
import arcgis
import pandas as pd
import numpy as np
import datetime
import requests
#import seaborn as sns
import matplotlib.pyplot as plt
#import rasterio
import pyodbc
import os
import warnings
import subprocess
import sys
#from shapely.geometry import Point
#import pyproj

GET DATA FROM POSTGIS

In [71]:
# Set up the PostgreSQL connection
connection = psycopg2.connect(host = '35.193.80.209',
                             database = 'Lab2',
                             user = 'postgres',
                             password = 'zyx11457')

cur = connection.cursor()
cur.execute("SELECT * FROM temperature_df")
rows = cur.fetchall()

#transform to pd dataframe and aggregate data
rows = pd.DataFrame(rows)
rows.columns = ['id','date','name','max_tmpf','min_tmpf','x','y','month']
rows = rows[rows['month']==3].reset_index(drop=True)

rows_agg = rows.groupby('id')[['max_tmpf','min_tmpf']].mean().reset_index()

rows = pd.merge(left = rows_agg, right=rows[['id','name','x','y']].drop_duplicates(), how='left',left_on=['id'], right_on=['id'])

In [80]:
arcpy.env.workspace = "E:/coursework/ARCGIS2/LAB3/LAB3"

sr = arcpy.SpatialReference(4326) # WGS 1984

# create a new feature class
arcpy.CreateFeatureclass_management(arcpy.env.workspace, "temperature", "POINT",spatial_reference=sr)

# add fields to the feature class
arcpy.AddField_management("E:/coursework/ARCGIS2/LAB3/LAB3/temperature.shp", "station_id", "Text")
#arcpy.AddField_management("E:/coursework/ARCGIS2/LAB3/LAB3/temperature.shp", "date", 'Date')
arcpy.AddField_management("E:/coursework/ARCGIS2/LAB3/LAB3/temperature.shp", "name", "Text")
arcpy.AddField_management("E:/coursework/ARCGIS2/LAB3/LAB3/temperature.shp", "max_tmpf", "Double")
arcpy.AddField_management("E:/coursework/ARCGIS2/LAB3/LAB3/temperature.shp", "min_tmpf", 'Double')
#arcpy.AddField_management("E:/coursework/ARCGIS2/LAB3/LAB3/temperature.shp", "month", "Short")

# insert data into the feature class
cursor = arcpy.da.InsertCursor("E:/coursework/ARCGIS2/LAB3/LAB3/temperature.shp", ["SHAPE@", "station_id", "name","max_tmpf",'min_tmpf'])
for index,row in rows.iterrows():
    point = arcpy.Point(row[4], row[5])
    cursor.insertRow([point, row[0], row[3],row[1], row[2]])
del cursor

'''
#another way to filter shp
input_shapefile = "temperature.shp"
output_shapefile = "temperature_March.shp"

sql_expression = "month = 3"
arcpy.Select_analysis(input_shapefile, output_shapefile, sql_expression)
'''

cur.close()
connection.close()

IDW INTERPORLATION

In [5]:
#IDW interporlation
arcpy.env.workspace = "E:/coursework/ARCGIS2/LAB3/LAB3"
input_fc = "temperature.shp"
geostat_layer =  "temperature_idw_geo.lyr"
output_raster = "temperature_idw.tif"

# Set up the IDW interpolation parameters
cell_size = 0.02 # Cell size of the output raster
power = 2 # Power parameter for IDW interpolation (optional, default is 2)

arcpy.CheckOutExtension("Spatial")
arcpy.ga.IDW(input_fc, 'max_tmpf',geostat_layer, output_raster,cell_size = cell_size, power = power)
#arcpy.sa.Idw(input_fc, "max_tmpf", output_raster, cell_size, power)

<Result 'temperature_idw_geo.lyr'>

KRIGING

In [6]:
#Bayesian Kriging
geostat_layer =  "temperature_BayesianKriging_geo.lyr"
output_raster = "temperature_BayesianKriging.tif"
arcpy.ga.EmpiricalBayesianKriging(input_fc, 'max_tmpf', geostat_layer, output_raster, cell_size = cell_size)
#arcpy.sa.Idw(input_fc, "max_tmpf", output_raster, cell_size, power)

<Result 'temperature_BayesianKriging_geo.lyr'>

RADIAL BASIS FUNCTIONS

In [7]:
#Radial Basis Functions
geostat_layer =  "temperature_RBF_geo.lyr"
output_raster = "temperature_RBF.tif"
arcpy.ga.RadialBasisFunctions(input_fc, 'max_tmpf', geostat_layer, output_raster, cell_size = cell_size)
#arcpy.sa.Idw(input_fc, "max_tmpf", output_raster, cell_size, power)

<Result 'temperature_RBF_geo.lyr'>

ASSESMENT (CROSS VALIDATION)

In [8]:
arcpy.CrossValidation_ga("temperature_idw_geo.lyr","temperature_idw_CV.shp")
arcpy.CrossValidation_ga("temperature_BayesianKriging_geo.lyr","temperature_BayesianKriging_CV.shp")
arcpy.CrossValidation_ga("temperature_RBF_geo.lyr","temperature_RBF_CV.shp")

<Result 'E:/coursework/ARCGIS2/LAB3/LAB3\\temperature_RBF_CV.shp'>

CALCULATE RMSE AND COMPARE

In [2]:
# Convert attribute table to a NumPy array and pd DataFrame
arcpy.env.workspace = "E:/coursework/ARCGIS2/LAB3/LAB3"

idw_cv_array = arcpy.da.FeatureClassToNumPyArray("temperature_idw_cv.shp", ["FID", "Error"])
idw_cv_pd = pd.DataFrame(idw_cv_array, columns=["FID", "Error"])

Kriging_cv_array = arcpy.da.FeatureClassToNumPyArray("temperature_BayesianKriging_cv.shp", ["FID", "Error"])
Kriging_cv_pd = pd.DataFrame(Kriging_cv_array, columns=["FID", "Error"])

RBF_cv_array = arcpy.da.FeatureClassToNumPyArray("temperature_RBF_cv.shp", ["FID", "Error"])
RBF_cv_pd = pd.DataFrame(RBF_cv_array, columns=["FID", "Error"])

# Calculate squared error
idw_cv_pd['squared_error'] = idw_cv_pd['Error'] ** 2
Kriging_cv_pd['squared_error'] = Kriging_cv_pd['Error'] ** 2
RBF_cv_pd['squared_error'] = RBF_cv_pd['Error'] ** 2

# Calculate RMSE
idw_rmse = numpy.sqrt(idw_cv_pd['squared_error'].mean())
Kriging_rmse = numpy.sqrt(Kriging_cv_pd['squared_error'].mean())
RBF_rmse = numpy.sqrt(RBF_cv_pd['squared_error'].mean())

#Print and compare results
print(f'The RMSE for IDW is: {idw_rmse}')
print(f'The RMSE for Kriging is: {Kriging_rmse}')
print(f'The RMSE for RBF is: {RBF_rmse}')

The RMSE for IDW is: 1.5042421137517141
The RMSE for Kriging is: 1.407298522569591
The RMSE for RBF is: 1.4526274613313865


In [4]:
# Set up the PostgreSQL connection
connection = psycopg2.connect(host = '35.193.80.209',
                             database = 'Lab3',
                             user = 'postgres',
                             password = 'zyx11457')

# Set the name of the new table in the PostgreSQL database
output_table = "temperature_assesment_table"

# Use psycopg2 to create the table in the PostgreSQL database
cursor = connection.cursor()
cursor.execute(f"CREATE TABLE {output_table} (method text, rmse double precision)")

# Use psycopg2 to copy the data from the Geodatabase table to the PostgreSQL table
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('IDW',idw_rmse))
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('Kriging',Kriging_rmse))
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('RBF',RBF_rmse))


# Commit the changes to the PostgreSQL database
connection.commit()

# Close the PostgreSQL connection and cursor
cursor.close()
connection.close()

Upload Interporlation Map (Kriging) to POSTGIS

In [4]:
#convert raster to point
arcpy.conversion.RasterToPoint(r"temperature_BayesianKriging.tif",'temperature_BayesianKriging_point.shp','VALUE')

#reproject
#outCS = arcpy.SpatialReference('NAD 1983 UTM Zone 15N')
#arcpy.management.Project("temperature_BayesianKriging_point.shp", r"temperature_BayesianKriging_point_reproject.shp", outCS)

#clip
arcpy.analysis.Clip("temperature_BayesianKriging_point.shp", r"shp_bdry_state/Boundaries_of_Minnesota.shp", r"temperature_BayesianKriging_point_clip.shp", None)

<Result 'E:\\coursework\\ARCGIS2\\LAB3\\LAB3\\temperature_BayesianKriging_point.shp'>

In [3]:
#upload using shp2pgsql
# Set the necessary parameters
shp2pgsql_path = r"D:\PostgreSQL\15\bin\shp2pgsql.exe"
psql_path = r"D:\PostgreSQL\15\bin\psql.exe"
shapefile_path = r"E:/coursework/ARCGIS2/LAB3/LAB3/temperature_BayesianKriging_point_clip.shp"
srid = 4326
schema_table = "temperature_BayesianKriging"
host = "35.193.80.209"
port = 5432
user = "postgres"
db_name = "Lab3"
password = "zyx11457"

# Build the shp2pgsql command
shp2pgsql_command = f"\"{shp2pgsql_path}\" -s {srid} {shapefile_path} {schema_table}"

# Build the psql command
psql_command = f"\"{psql_path}\" -h {host} -p {port} -U {user} -d {db_name} -w"

# Combine the commands using a pipe
full_command = f"{shp2pgsql_command} | {psql_command}"

# Set up the environment with the password
env = os.environ.copy()
env["PGPASSWORD"] = password

# Run the command using the subprocess module and capture stderr
process = subprocess.Popen(full_command, shell=True, env=env, stderr=subprocess.PIPE)

# Wait for the process to finish and capture the return code and stderr
return_code = process.wait()
stderr_output = process.stderr.read().decode()

# Print the return code and error message
print(f"Return code: {return_code}")
print(f"Error message: {stderr_output}")

Return code: 0
Error message: could not print result table: Invalid argument
Field pointid is an FTDouble with width 10 and precision 0
Field grid_code is an FTDouble with width 13 and precision 11
Shapefile type: Point
Postgis type: POINT[2]



Download geojson from url using flask

In [4]:
import requests
import json
import os

# Replace this URL with the address of your Flask application
url = "http://34.125.39.137:5000/temperature"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    geojson_data = response.json()

    # Save the GeoJSON to the specified folder and name it "lab3_test.geojson"
    output_folder = r"E:/coursework/ARCGIS2/LAB3/LAB3"
    output_file_path = os.path.join(output_folder, "temperature_inteporlation.geojson")
    print(f"Output file path: {output_file_path}")

    with open(output_file_path, "w") as f:
        json.dump(geojson_data, f)
    print(f"GeoJSON file saved as {output_file_path}")
else:
    print("Error fetching GeoJSON data")


Output file path: E:/coursework/ARCGIS2/LAB3/LAB3\temperature_inteporlation.geojson
GeoJSON file saved as E:/coursework/ARCGIS2/LAB3/LAB3\temperature_inteporlation.geojson


Convert geojson into esri geojson

In [5]:
import json

# Read input JSON data from file
with open(r'E:/coursework/ARCGIS2/LAB3/LAB3/temperature_inteporlation.geojson', 'r') as input_file:
    input_data = json.load(input_file)

# Process input data and create output data
output_data = {
    "type":"FeatureCollection",
    "features":[
        {
            "type":"Feature",
            "geometry":{
                "type":"Point",
                "coordinates":point_data["geometry"]["coordinates"],
            },
            "properties":{"max_tmpf": point_data["grid_code"]},
        } for point_data in input_data
    ] 
}

# Write output JSON data to file
with open(r'E:/coursework/ARCGIS2/LAB3/LAB3/temperature_inteporlation_esri.geojson', 'w') as output_file:
    json.dump(output_data, output_file, indent=2)

UPLOAD TO ARCGIS ONLINE

In [7]:
import arcgis
gis = arcgis.gis.GIS("home")

# File previously loaded into AGOL (manually)
file_path = r"E:/coursework/ARCGIS2/LAB3/LAB3/temperature_inteporlation_esri.geojson"

# Add item 
props = {"title":"temperature_inteporlation", "description":"Lab 3", "tags":"GIS 5572", "type":"GeoJson"}
item = gis.content.add(item_properties=props, data=file_path)

# Publish item
item.publish(None)

<Item title:"temperature_inteporlation" type:Feature Layer Collection owner:zhan6322_UMN>

In [28]:
#Json to feature class
arcpy.conversion.JSONToFeatures("E:/coursework/ARCGIS2/LAB3/LAB3/temperature_inteporlation_esri.geojson", "E:/coursework/ARCGIS2/LAB3/LAB3/temperature_inteporlation_feature1.shp", "POINT")

#upload feature class to arconline
import zipfile
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Function to zip shapefile
def zip_shapefile(shapefile_path):
    with zipfile.ZipFile(shapefile_path + '.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
        for file_name in os.listdir(os.path.dirname(shapefile_path)):
            if file_name.startswith(os.path.basename(shapefile_path).split('.')[0]) and not file_name.endswith('.zip'):
                zf.write(os.path.join(os.path.dirname(shapefile_path), file_name), arcname=file_name)

# Connect to your ArcGIS Online organization
gis = GIS("home")

# Set the input shapefile path
input_shapefile = r'E:/coursework/ARCGIS2/LAB3/LAB3/temperature_inteporlation.shp'

# Zip the shapefile
zip_shapefile(input_shapefile)

# Upload the zipped shapefile as a new item
item = gis.content.add({'type': 'Shapefile'}, input_shapefile + '.zip')

# Publish the new item as a feature layer
feature_layer_item = item.publish()

# Get the feature layer URL and print it
feature_layer_url = feature_layer_item.url
print(feature_layer_url)

<Result 'E:\\coursework\\ARCGIS2\\LAB3\\LAB3\\temperature_inteporlation_feature1.shp'>

Upload Interporlation Difference (Kriging) Layer to POSTGIS

In [11]:
#upload using shp2pgsql
# Set the necessary parameters
shp2pgsql_path = r"D:\PostgreSQL\15\bin\shp2pgsql.exe"
psql_path = r"D:\PostgreSQL\15\bin\psql.exe"
shapefile_path = r"E:/coursework/ARCGIS2/LAB3/LAB3/temperature_BayesianKriging_CV.shp"
srid = 4326
schema_table = "temperature_BayesianKriging_difference"
host = "35.193.80.209"
port = 5432
user = "postgres"
db_name = "Lab3"
password = "zyx11457"

# Build the shp2pgsql command
shp2pgsql_command = f"\"{shp2pgsql_path}\" -s {srid} {shapefile_path} {schema_table} -c error:double precision"

# Build the psql command
psql_command = f"\"{psql_path}\" -h {host} -p {port} -U {user} -d {db_name} -w"

# Combine the commands using a pipe
full_command = f"{shp2pgsql_command} | {psql_command}"

# Set up the environment with the password
env = os.environ.copy()
env["PGPASSWORD"] = password

# Run the command using the subprocess module and capture stderr
process = subprocess.Popen(full_command, shell=True, env=env, stderr=subprocess.PIPE)

# Wait for the process to finish and capture the return code and stderr
return_code = process.wait()
stderr_output = process.stderr.read().decode()

# Print the return code and error message
print(f"Return code: {return_code}")
print(f"Error message: {stderr_output}")

# Set up the PostgreSQL connection
connection = psycopg2.connect(host = '35.193.80.209',
                             database = 'Lab3',
                             user = 'postgres',
                             password = 'zyx11457')

# Set the name of the new table in the PostgreSQL database
output_table = "temperature_BayesianKriging_difference"

# Use psycopg2 to create the table in the PostgreSQL database
cursor = connection.cursor()
cursor.execute(f"ALTER TABLE {output_table} ALTER COLUMN error SET DATA TYPE double precision;")
#cursor.execute(f"")

# Commit the changes to the PostgreSQL database
connection.commit()

# Close the PostgreSQL connection and cursor
cursor.close()
connection.close()

Return code: 0
Error message: could not print result table: Invalid argument
Field measured is an FTDouble with width 19 and precision 11
Field predicted is an FTDouble with width 19 and precision 11
Field error is an FTDouble with width 19 and precision 11
Field stderror is an FTDouble with width 19 and precision 11
Field stdd_error is an FTDouble with width 19 and precision 11
Field normvalue is an FTDouble with width 19 and precision 11
Field crps is an FTDouble with width 19 and precision 11
Field quanval is an FTDouble with width 19 and precision 11
Field source_id is an FTDouble with width 10 and precision 0
Shapefile type: Point
Postgis type: POINT[2]



Download geojson from url using flask

In [18]:
import requests
import json
import os

# Replace this URL with the address of your Flask application
url = "http://34.125.39.137:5000/tmp_difference"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    geojson_data = response.json()

    # Save the GeoJSON to the specified folder and name it "lab3_test.geojson"
    output_folder = r"E:/coursework/ARCGIS2/LAB3/LAB3"
    output_file_path = os.path.join(output_folder, "temperature_inteporlation_difference.geojson")
    print(f"Output file path: {output_file_path}")

    with open(output_file_path, "w") as f:
        json.dump(geojson_data, f)
    print(f"GeoJSON file saved as {output_file_path}")
else:
    print("Error fetching GeoJSON data")

Output file path: E:/coursework/ARCGIS2/LAB3/LAB3\temperature_inteporlation_difference.geojson
GeoJSON file saved as E:/coursework/ARCGIS2/LAB3/LAB3\temperature_inteporlation_difference.geojson


In [19]:
import json

# Read input JSON data from file
with open(r'E:/coursework/ARCGIS2/LAB3/LAB3/temperature_inteporlation_difference.geojson', 'r') as input_file:
    input_data = json.load(input_file)

# Process input data and create output data
output_data = {
    "type":"FeatureCollection",
    "features":[
        {
            "type":"Feature",
            "geometry":{
                "type":"Point",
                "coordinates":point_data["geometry"]["coordinates"],
            },
            "properties":{"error": point_data["error"]},
        } for point_data in input_data
    ] 
}

# Write output JSON data to file
with open(r'E:/coursework/ARCGIS2/LAB3/LAB3/temperature_inteporlation_difference_esri.geojson', 'w') as output_file:
    json.dump(output_data, output_file, indent=2)

UPLOAD TO ARCGIS ONLINE

In [20]:
import arcgis
gis = arcgis.gis.GIS("home")

# File previously loaded into AGOL (manually)
file_path = r"E:/coursework/ARCGIS2/LAB3/LAB3/temperature_inteporlation_difference_esri.geojson"

# Add item 
props = {"title":"temperature_inteporlation_difference", "description":"Lab 3", "tags":"GIS 5572", "type":"GeoJson"}
item = gis.content.add(item_properties=props, data=file_path)

# Publish item
item.publish(None)

<Item title:"temperature_inteporlation_difference" type:Feature Layer Collection owner:zhan6322_UMN>